In [ ]:
df = pandas.read_csv('dataset-solar/Plant_1_Generation_Data.csv')

In [ ]:
print(df)

In [ ]:
df['DATE'],df['TIME']=df['DATE_TIME'].str.split(' ',1).str

In [ ]:
df

In [ ]:
del df['DATE_TIME']
df

In [ ]:
df = df[ ['TIME'] + [ col for col in df.columns if col != 'TIME' ] ]

In [ ]:
df = df[ ['DATE'] + [ col for col in df.columns if col != 'DATE' ] ]

In [ ]:
df

In [ ]:
del df['TOTAL_YIELD']

In [ ]:
df

In [ ]:
del df['DAILY_YIELD']
df

In [ ]:
df.duplicated(['DATE', 'TIME'])

In [ ]:
df.groupby(['DATE','TIME'], as_index=False).mean()

In [ ]:
df = df.groupby(['DATE','TIME'], as_index=False).mean()

In [ ]:
df

In [ ]:
df_gen = df

In [ ]:
df_wet = pandas.read_csv('dataset-solar\Plant_1_Weather_Sensor_Data.csv')

In [ ]:
df_wet

In [ ]:
del df_wet['PLANT_ID']
df_wet

In [ ]:
del df_wet['SOURCE_KEY']
df_wet

In [33]:
df_wet['DATE'],df_wet['TIME']=df_wet['DATE_TIME'].str.split(' ',1).str
df_wet

,DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,DATE,TIME
0,2020-05-15 00:00:00,25.184316,22.857507,0.0,2020-05-15,00:00:00
1,2020-05-15 00:15:00,25.084589,22.761668,0.0,2020-05-15,00:15:00
2,2020-05-15 00:30:00,24.935753,22.592306,0.0,2020-05-15,00:30:00
3,2020-05-15 00:45:00,24.846130,22.360852,0.0,2020-05-15,00:45:00
4,2020-05-15 01:00:00,24.621525,22.165423,0.0,2020-05-15,01:00:00
...,...,...,...,...,...,...
3177,2020-06-17 22:45:00,22.150570,21.480377,0.0,2020-06-17,22:45:00
3178,2020-06-17 23:00:00,22.129816,21.389024,0.0,2020-06-17,23:00:00
3179,2020-06-17 23:15:00,22.008275,20.709211,0.0,2020-06-17,23:15:00
3180,2020-06-17 23:30:00,21.969495,20.734963,0.0,2020-06-17,23:30:00


In [34]:
df_wet = df_wet[ ['TIME'] + [ col for col in df_wet.columns if col != 'TIME' ] ]
df_wet = df_wet[ ['DATE'] + [ col for col in df_wet.columns if col != 'DATE' ] ]
df_wet

,DATE,TIME,DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15,00:00:00,2020-05-15 00:00:00,25.184316,22.857507,0.0
1,2020-05-15,00:15:00,2020-05-15 00:15:00,25.084589,22.761668,0.0
2,2020-05-15,00:30:00,2020-05-15 00:30:00,24.935753,22.592306,0.0
3,2020-05-15,00:45:00,2020-05-15 00:45:00,24.846130,22.360852,0.0
4,2020-05-15,01:00:00,2020-05-15 01:00:00,24.621525,22.165423,0.0
...,...,...,...,...,...,...
3177,2020-06-17,22:45:00,2020-06-17 22:45:00,22.150570,21.480377,0.0
3178,2020-06-17,23:00:00,2020-06-17 23:00:00,22.129816,21.389024,0.0
3179,2020-06-17,23:15:00,2020-06-17 23:15:00,22.008275,20.709211,0.0
3180,2020-06-17,23:30:00,2020-06-17 23:30:00,21.969495,20.734963,0.0


In [35]:
del df_wet['DATE_TIME']
df_wet

,DATE,TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15,00:00:00,25.184316,22.857507,0.0
1,2020-05-15,00:15:00,25.084589,22.761668,0.0
2,2020-05-15,00:30:00,24.935753,22.592306,0.0
3,2020-05-15,00:45:00,24.846130,22.360852,0.0
4,2020-05-15,01:00:00,24.621525,22.165423,0.0
...,...,...,...,...,...
3177,2020-06-17,22:45:00,22.150570,21.480377,0.0
3178,2020-06-17,23:00:00,22.129816,21.389024,0.0
3179,2020-06-17,23:15:00,22.008275,20.709211,0.0
3180,2020-06-17,23:30:00,21.969495,20.734963,0.0
